In [2]:
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import time
import math
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader

In [177]:
class DendriticGatedNet(nn.Module):
    def __init__(self, n_inputs, n_hiddens, n_out):
        super().__init__()
        self.n_inputs = n_inputs
        self.n_hiddens = n_hiddens
        self.n_out = n_out
        
        self.layer1 = DendriticGatedLayer(
            n_units=n_hiddens,
            n_prev_layer_units=n_inputs,
            n_network_inputs=n_inputs
        )
        
        self.layer2 = DendriticGatedLayer(
            n_units=n_hiddens,
            n_prev_layer_units=n_hiddens,
            n_network_inputs=n_inputs
        )
        
        self.outlayer = nn.Linear(n_hiddens, n_out)
    
    def forward(self, X):
        '''
            TODO
        '''
        None
    

class DendriticGatedLayer(nn.Module):
    def __init__(self, n_units, n_prev_layer_units, n_network_inputs):
        super().__init__()
        self.weights = nn.Parameter(torch.rand(n_units, n_prev_layer_units), requires_grad=True)
        #self.halfplane_gates = torch.normal(torch.Tensor([[0]*n_units]*n_network_inputs)).T
        self.halfplane_gates = torch.randn_like(torch.zeros(n_units,n_prev_layer_units,n_network_inputs))
        
        self.n_units = n_units
        self.n_prev_layer_units = n_prev_layer_units
        self.n_network_inputs = n_network_inputs
        
        
    def forward(self, X, network_inputs, gate_threshold=0):
        gating = (d.halfplane_gates @ network_inputs.T)
        gate_states = torch.permute(gating, (2,0,1)) > 0
        active_weights = torch.mul(self.weights.repeat([X.shape[0],1,1]), gate_states)
        output = (active_weights @ X.T)[0].T
        
        '''
            self.gate_states is now a (n_units x n_prev_layer_units) array.
            Each row gives the gate state of the afferent synapses of a single unit in the current layer
        '''
        
        return output
        

In [174]:
d = DendriticGatedLayer(10,5,2)

In [175]:
inp = torch.rand(4,5)
network_inputs = torch.rand(4,2)

In [176]:
d.forward(inp, network_inputs)

tensor([[0.2282, 1.2714, 0.0400, 0.0720, 0.7430, 1.0352, 0.4621, 0.0389, 0.1746,
         0.1124],
        [0.1787, 0.8203, 0.4011, 0.6507, 0.3355, 0.4218, 0.2907, 0.4208, 0.1629,
         0.1397],
        [0.2764, 1.4663, 0.3093, 0.6671, 1.0366, 1.0819, 0.8055, 0.3213, 0.4716,
         0.3986],
        [0.3226, 1.0308, 0.4489, 1.3911, 1.1897, 0.6068, 1.1889, 0.4685, 1.0649,
         1.0100]], grad_fn=<PermuteBackward0>)